## Class Object to downlaod and parse IRS 990 forms

In [ ]:
import os, sys
# configure env for IRSx
os.environ["IRSX_CACHE_DIRECTORY"] = './IRSX_CACHE/'
os.environ["IRSX_INDEX_DIRECTORY"] = './IRSX_CACHE/'
os.environ["IRSX_WORKING_DIRECTORY"] = './IRSX_CACHE/'

import json
import pprint
import numpy as np
import pandas as pd
import urllib.request
from IPython.display import display, HTML
import xml.etree.cElementTree as et
from irsx.xmlrunner import XMLRunner
from collections import defaultdict

In [ ]:
class AWSparser():
    def __init__( self, config=False ):
    
        self.xml_runner = XMLRunner()
        self.variables = defaultdict(dict)
        self.variable_names = dict()
        self.validVariables = set()
        
        if config:
            # Read configuration file from disk
            #print( 'open config: %s' % config )
            with open(config) as fp:  
                for line in fp:
                    if line[0] == '#':
                        pass
                    else:
                        self.validVariables.add( line.strip() )

        # Prep definitions of variables from IRSx module
        for test in  self.xml_runner.standardizer.variables:
            each = self.xml_runner.standardizer.variables[test]
            self.variables[ each['db_table'] ][ each['db_name'] ] = test
            
        # Make list on final names of variables
        for x in self.variables:
            for y in self.variables[x]:
                self.variable_names[ self.variables[x][y] ] = { 
                    'table':x, 
                    'field':y, 
                    'use':  self.variables[x][y] in self.validVariables 
                }
            
    def list_variables( self, name=False ):
        '''
        Print list of valid variable names. just the final representation
        of them. Not the original object names on the form.
        '''
        if name and name in self.variable_names:
            return self.variable_names[ name ]
        
        return sorted(set(self.variable_names.keys()))

In [ ]:
class Filing(AWSparser):
    def __init__( self, config=False, **metadata ):
    
        AWSparser.__init__(self, config)
        
        self.activeRaw = None
        self.active = dict()
        self.filePath = None
        self.id = None
        self.ein = None
        self.dln = None
        self.url = None
        self.formType = None
        self.taxPeriod = None
        self.lastUpdated = None
        self.submittedOn = None
        self.organization = None
        self.metadata = dict()
        
        if metadata:
            self.setMetadata( metadata )
            self.get()
            
            
    def setMetadata( self, info ):
        '''
        Populate the metadata of this document
        '''
        
        self.metadata = info
        
        self.ein = info['EIN']
        self.dln = info['DLN']
        self.url = info['URL']
        self.id = info['ObjectId']
        self.formType = info['FormType']
        self.taxPeriod = info['TaxPeriod']
        self.lastUpdated = info['LastUpdated']
        self.submittedOn = info['SubmittedOn']
        self.organization = info['OrganizationName']
        
        # Example:
        #      EIN                 042662873
        #      TaxPeriod           201603
        #      DLN                 93493243000066
        #      FormType            990
        #      URL                 https://s3.amazonaws.com/irs-form-990/...
        #      OrganizationName    ELKS BUILDING CORP OF NORWOOD
        #      SubmittedOn         2017-01-04
        #      ObjectId            201612439349300006
        #      LastUpdated         2017-01-11T22:15:15
        
        
    def display( self ):
        '''
        Nice (native pandas) print of the DataFrame 990 data
        
        '''
        
        if not len(self.active):
            print( 'ERROR: Internal cache is empty!')
            return
        
        display( self.active )
        
    
    def __str__(self):
        '''
        Metadata representation of the object.
        '''
        
        text =  '''
        EIN         [%s]
        Name        [%s]
        TaxPeriod   [%s]
        DLN         [%s]
        FormType    [%s]
        SubmittedOn [%s]
        ObjectId    [%s]
        LastUpdated [%s]
        [%s]
        ''' % ( self.ein, self.organization, self.taxPeriod, self.dln, 
        self.formType, self.submittedOn, self.id, self.lastUpdated, self.url )
        
        return text
    
    def __del__(self):
        # Remove temp file if possible
        try:
            os.remove( self.filePath )
        except:
            pass
        
    def __repr__(self):
        return str(pprint.pprint(self.active))

    def __getitem__(self, item):
        if item in self.active:
            return self.active[item]
        else:
            return None
    
    def info( self ):
        return self.__str__()
    
    def _getRaw( self ):
        '''
        Download the actual 990 form from AWS using IRSx tools
        '''
        self.activeRaw = self.xml_runner.run_filing( self.id, verbose=False )
        
    def json( self ):
        '''
        Convert to JSON format. Needed for MongoDB uploads
        '''
        return json.dumps( self.active )
        
    def get( self, **metadata ):
        '''
        Download the actual 990 form from AWS using IRSx tools
        '''
        
        if metadata:
            self.setMetadata( metadata )
        
        #print( 'Get Form: [%s]' % self.id )
        self._getRaw()
        
        # save path of temp file for cleanup later
        self.filePath = self.activeRaw.get_filepath()
        
        # convert to normalized dict
        self.parse()
        
        self.active.update(self.metadata)
    
    def list_schedules( self ): 
        '''
        Return a list of all valid segments on the active form.
        '''
        
        if self.activeRaw:
            return self.activeRaw.list_schedules()
        else:
            print( 'ERROR: No active form selected!' )
            return None
        
    def parse( self ):
        '''
        Convert active object to final flat format.
        '''
        
        if self.activeRaw:
            temp = dict()
            for sked in self.activeRaw.get_result():
                #print("Segment: %s" % sked['schedule_name'])  
                for x in sked['schedule_parts']:
                    for y in sked['schedule_parts'][x]:
                        
                        # verify if we set the value off
                        if x in self.variables and y in self.variables[x] \
                            and not self.variable_names[ self.variables[x][y] ]['use']:
                            continue
                            
                        # try to flatten the name
                        try:
                            # clean a bit
                            if not sked['schedule_parts'][x][y]: continue
                            if sked['schedule_parts'][x][y] == 'false': continue
                                
                            # one level only
                            temp[ self.variables[x][y] ] = sked['schedule_parts'][x][y]
                            
                        except:
                            if not y in ['object_id', 'ein']:
                                print( 'ERROR on %s.%s.%s' % (sked['schedule_parts'], x, y) )
                                
                                
            # replace original object
            self.active = temp
                    
        else:
            self.active = dict()
            print( 'ERROR: No active form selected!' )

In [ ]:
class classy990forms(AWSparser):
    def __init__( self, year=False, config=False, debug=False ):
    
        AWSparser.__init__(self, config)
        
        self.index = 0
        self.config = config
        self.debug = debug
        self.set_year = year
        self.sourceURL = False
        self.pkl_name = False
        self.df = pd.DataFrame()
        
        if year:
            self.year( year )
        
        
    def year( self, year, reset_index=False ):
        self.set_year = year
        self.reset_index = reset_index
        
        self.sourceURL = 'https://s3.amazonaws.com/irs-form-990/index_%d.json' % self.set_year
        self.pkl_name = "./IRS990_%d.pkl" % self.set_year

        if self.debug: print( 'Working on year: %d' % self.set_year)
        if self.debug: print( 'index URL: %d' % self.set_year)
        
        self.load_index()
    
    def load_index( self, reset_index=False ):
        
        if not self.sourceURL or not self.pkl_name:
            print( 'ERROR: Neet to configure object.year(####) first.')
            return
        
        if self.debug: print( 'Prepare INDEX on year: %d' % self.set_year)
            
        # Maybe we want to reset the local cache
        if self.reset_index and os.path.isfile( self.pkl_name ):
            if self.debug: print( 'Remove previous cached INDEX: %d' % self.pkl_name)
            try:
                os.remove( self.pkl_name )
            except:
                self.print( 'ERROR: Cannot remove previous index: %s' % self.pkl_name )
                return
            
            self.reset_index = False
            
        # Verify if we already have the file
        if os.path.isfile( self.pkl_name ):
            # Read from disk
            if self.debug: 
                print( 'Found local copy: %s' % self.pkl_name)
                sys.stdout.write('LOADING...\r')
                sys.stdout.flush()
            self.df = pd.read_pickle( self.pkl_name )
        else:
            # Download from URL
            if self.debug:
                print( 'Download new index from: %s' % self.sourceURL)
                sys.stdout.write('DOWNLOADING...\r')
                sys.stdout.flush()
            self.df = pd.read_json( self.sourceURL )
            if self.debug:
                sys.stdout.write('PARSING DOWNLOADED FILE...\r')
                sys.stdout.flush()
            self.df = self.df[self.df.keys()[0]].apply(lambda x:pd.Series(x))
            # Save version to disk
            if self.debug:
                sys.stdout.write('SAVING COPY TO DISK...\r')
                sys.stdout.flush()
            self.df.to_pickle( self.pkl_name )
            if self.debug: print( 'New cached form: %s' % self.pkl_name)

        if self.debug: print( self.df )
            
    
    def display( self ):
        '''
        Nice (native pandas) print of the DataFrame 990 data
        
        '''
        
        if not len(self.df):
            print( 'ERROR: Internal cache is empty!')
            return
        
        display( self.df )
    
    def __str__(self):
        '''
        String representation of the 990 form data
        To be called with the print on str methods:
            print(form)
        '''
        return str( self.df )
    
    def __repr__(self):
        '''
        Simple representation of the object.
            print( form.info() )
            form.info()
            or just "form" as the last line in notebook cell
        '''
        
        return 'IRS 990 form for year [%s]: %d documents' % (self.set_year, len(self.df) )

    def info( self ):
        return self.__repr__()
    
    def get( self, id=False, ein=False ):
        '''
        Download the actual 990 form from AWS using IRSx tools
        '''
        
        if id:
            variable = 'ObjectId'
            value = id
        elif ein:
            variable = 'EIN'
            value = ein
        else:
            print('ERROR: Only [ObjectId] or [EIN]')
            return None
        
        
        if self.debug: print( 'Downlaod %s:%s' % (variable,value) )
            
        # The identification of the row changes from iloc to loc
        return Filing(config=self.config, **self.df.loc[self.df[variable] == value].to_dict('records')[0] )
    
    # Need iterator
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            result =  Filing(config=self.config, **self.df.iloc[self.index].to_dict() )
        except IndexError:
            raise StopIteration
        self.index += 1
        return result